In [4]:
from copy import deepcopy
import json

data = open("AoC2021-18.input").read()
data = [json.loads(x) for x in data.strip().splitlines()]

def magnitude(pair):
    if isinstance(pair, list):
        if len(pair) > 1:
            return magnitude(pair[0]) * 3 + magnitude(pair[1]) * 2
        else:
            return magnitude(pair[0])
    else:
        return pair

def split(pair):
    lst = walk(pair)
    if any(x[0]>9 for x in lst):
        ret = []
        done = False
        for x in  lst:
            if done or x[0]<10:
                ret.append(x)
            else:
                ret.append([x[0]//2, x[1]+1])
                ret.append([(x[0]+1)//2, x[1]+1])
                done = True
        return unwalk(ret), True
    else:
        return pair, False

def walk(pair, deep=-1):
    if isinstance(pair, list):
        if len(pair) > 1:
            return walk(pair[0], deep + 1)+walk(pair[1], deep + 1)
        else:
            return walk(pair[0], deep + 1)
    else:
        return [[pair, deep]]

def unwalk(lst, deep=0):
    ret = []
    lst = lst[:]
    while lst and lst[0][1]>=deep:
        if deep == lst[0][1]:
            ret.append(lst[0][0])
            lst = lst[1:]
        else:
            subret, lst = unwalk(lst, deep+1)
            ret.extend(subret)
    if len(ret)>1:
        ret = [[x, y] for x, y in zip(ret[::2], ret[1::2])]
    else:
        ret = [ret]
    if deep:
        return ret, lst
    else:
        return ret[0]


def explode(pair, deep=0):
    lst = walk(pair)
    if not any(x[1]==4 for x in lst):
        return pair, False

    ret = []
    second = False
    skip = False
    right = None
    for x in lst:
        if skip:
            if right is None:
                ret.append(x[:])
            else:
                ret.append([x[0]+right, x[1]])
                right = None
            continue
        if x[1] == 4:
            if second:
                right = x[0]
                skip = True
            else:
                if ret:
                    ret[-1] = [ret[-1][0] + x[0], ret[-1][1]]
                second = True
                ret.append([0, x[1]-1])
        else:
            ret.append(x[:])
    return unwalk(ret), True

def reducesn(pair):
    ret = deepcopy(pair)
    while True:
        ret, didit = explode(ret)
        if didit:
            continue
        ret, didit = split(ret)
        if didit:
            continue
        break
    return ret
    
def xreducesn(pair):
    ret = deepcopy(pair)
    oops = True
    while oops:
        oops = False
        while True:
            ret, didit = explode(ret)
            if not didit:
                break
            oops = True
        while True:
            ret, didit = split(ret)
            if not didit:
                break
            oops = True
    return ret


def zreducesn(pair):
    ret = deepcopy(pair)
    while True:
        ret, didit = explode(ret)
        if didit:
            print('*', end='')
            continue
        ret, didit = split(ret)
        if didit:
            print('/', end='')
            continue
        break
    return ret

def addnums(pair1, pair2):
    ret = [deepcopy(pair1), deepcopy(pair2)]
    return reducesn(ret)

def xaddnums(pair1, pair2):
    ret = [deepcopy(pair1), deepcopy(pair2)]
    return xreducesn(ret)

print(max(magnitude(addnums(x,y)) for x in data for y in data if x != y))


4650
